In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from playsound import playsound

In [7]:
def notif(a,b):
    ImgFileName="output_image.jpg"
    with open(ImgFileName, 'rb') as f:
        img_data = f.read()

    msg = MIMEMultipart()
    msg['Subject'] = 'ALERT!'
    msg['From'] = 'mohamedfazith510@gmail.com'
    msg['To'] = 'wilddetec@gmail.com'
    text = MIMEText("Wild Animal Detected :\nType : "+a+"\nPercentage :"+b[0:5]+"%")
    msg.attach(text)
    image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
    msg.attach(image)
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('mohamedfazith510@gmail.com','Mdfazith@01')
    s.sendmail('mohamedfazith510@gmail.com','wilddetec@gmail.com', msg.as_string())
    s.quit()

def classification(a,b):
    file = open("animals.txt", "r")
    line=file.read()
    if (a in line):
#     animal=["lion","tiger","cheetah","timber wolf", "grey wolf", "gray wolf"]
#     if(a in animal):
        if(b>40):
            c=str(b)
            notif(a,c)
            print("Wild animal alert!!!")
            print(a,"{:.2f}%".format(b))
            aud_file="C:\\Users\\Asus\\Wildlife detection\\alarm1.mp3"
            playsound(aud_file)

def detection():    
    file='segmented_object_1.jpg'
    c=1
    while(os.path.exists('segmented_object_'+str(c)+'.jpg')):
        file='segmented_object_'+str(c)+'.jpg'
        import cv2 
        imgg=cv2.imread(file)
        imgg=cv2.resize(imgg,(224,224))   
#     plt.imshow(cv2.cvtColor(imgg,cv2.COLOR_BGR2RGB))
        from tensorflow.keras.preprocessing import image
        mobile=tf.keras.applications.mobilenet_v2.MobileNetV2()
        resized_img= image.img_to_array(imgg)
        final_image= np.expand_dims(resized_img,axis=0)
        final_image=tf.keras.applications.mobilenet.preprocess_input(final_image)
        from tensorflow.keras.applications import imagenet_utils
        predictions=mobile.predict(final_image)
        results= imagenet_utils.decode_predictions(predictions)
        #print(results)
        a=results[0][0][1]
        b=results[0][0][2]*100
        classification(a,b)
        c+=1
    
cam=cv2.VideoCapture("tiger_video_2.mp4")
#namedWindow("camera")
global count
count =0
    
while True:
    
    ret, frame = cam.read()
    if not ret:
        break
#     cv2.imshow("camera",frame)
    cv2.imwrite('./frames/'+str(count)+'.jpg',frame)
    
    import pixellib
    from pixellib.torchbackend.instance import instanceSegmentation
    ins = instanceSegmentation()
    ins.load_model("pointrend_resnet50.pkl")
    filename='frames/'+str(count)+'.jpg'
    ins.segmentImage(filename, show_bboxes=True, extract_segmented_objects=True,
    save_extracted_objects=True, output_image_name="output_image.jpg" )
    detection()
    if cv2.waitKey(10)==27:
        break
    
    count+=1
           
cam.release()
cv2.destroyAllWindows()

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


Wild animal alert!!!
tiger 51.35%


KeyboardInterrupt: 